In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import maxjoshua as mh
import tensorflow as tf
import sklearn.preprocessing

## Load dataset

In [2]:
import sklearn.datasets
X, y = sklearn.datasets.make_regression(n_samples=1000, n_features=100, n_informative=20, n_targets=3)

In [3]:
X.shape, y.shape, y.max(), y.min()

((1000, 100), (1000, 3), 957.640418472872, -849.9321891784462)

## Feature selection

In [4]:
%%time 
indices, values, num_in, num_out = mh.pretrain_submodels(
    sklearn.preprocessing.scale(X), 
    sklearn.preprocessing.scale(y), 
    num_out=64, n_select=3)

0 (42, 53, 77) [0.41072096042140604, 0.3169109462233002, -0.02538806349456914] 0.8503307593845821
1 (28, 44, 71) [0.3314903653203117, 0.12272168714932445, 0.3677154743419552] 0.8385466172243783
2 (13, 45, 81) [0.058972471965477766, 0.3035680522122819, 0.347724698889192] 0.8957429767433448
CPU times: user 22.1 s, sys: 750 ms, total: 22.9 s
Wall time: 28.6 s


## Training

In [5]:
model = tf.keras.models.Sequential([
    mh.SparseLayerAsEnsemble(
        num_in=num_in, 
        num_out=num_out, 
        sp_indices=indices, 
        sp_values=values,
        sp_trainable=False,
        norm_trainable=True,
    ),
    tf.keras.layers.Dense(
        units=3, use_bias=False,
        # kernel_constraint=tf.keras.constraints.NonNeg()
    ),
    mh.InverseTransformer(
        units=3,
        init_bias=y.mean(), 
        init_scale=y.std()
    )
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=3e-4, beta_1=.9, beta_2=.999, epsilon=1e-7, amsgrad=True),
    loss='mean_squared_error'
)

history = model.fit(X, y, epochs=3)

2022-06-22 15:30:26.412520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/3
The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'scale:0' shape=(3,) dtype=float32>
  <tf.Variable 'bias:0' shape=(3,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
32/32 [==============================] - 2s 6ms/step - loss: 43438.5469
Epoch 2/3
32/32 [==============================] - 0s 6ms/step - loss: 39415.8281
Epoch 3/3
32/32 [==============================] - 0s 6ms/step - loss: 36680.3125


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sparse_layer_as_ensemble (S  (None, 64)               192       
 parseLayerAsEnsemble)                                           
                                                                 
 dense (Dense)               (None, 3)                 192       
                                                                 
 inverse_transformer (Invers  (None, 3)                6         
 eTransformer)                                                   
                                                                 
Total params: 390
Trainable params: 198
Non-trainable params: 192
_________________________________________________________________


In [7]:
y_pred = model.predict(X)

32/32 [==============================] - 0s 3ms/step


In [8]:
y_pred

array([[-607.3181   , -450.62985  , -405.91162  ],
       [-281.22302  , -281.6296   , -297.2379   ],
       [ -70.37455  ,  -60.6155   , -116.41666  ],
       ...,
       [ 365.05472  ,  332.31036  ,  186.08226  ],
       [ 276.50476  ,  188.53299  ,   63.09299  ],
       [  -3.7496881,  -62.109215 , -129.27966  ]], dtype=float32)

In [9]:
y

array([[-475.14647091, -362.47518431, -306.34891834],
       [-408.29456091, -487.32369026, -394.02080307],
       [  -7.40124639, -345.13785761, -530.26698008],
       ...,
       [ 532.50899026,  113.68553711,  186.2086209 ],
       [ 119.38910374,   48.26568551,  105.12092451],
       [  66.26992169, -216.74575749, -182.41523742]])

In [11]:
# model.save("test-save")
# model2 = tf.keras.models.load_model("test-save")
# y_pred2 = model2.predict(X)
# y_pred == y_pred2

INFO:tensorflow:Assets written to: test-save/assets


INFO:tensorflow:Assets written to: test-save/assets


32/32 [==============================] - 0s 3ms/step


array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       ...,
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])